In [1]:
from sklearn.preprocessing import OneHotEncoder

import pandas as pd
import numpy as np
import seaborn as sns # data visualization library  
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score,precision_score,recall_score,f1_score, roc_auc_score

In [2]:
df = pd.read_csv('../data/crawler/unified-events-statistics.csv')
df = df.drop(['golId','game','event4','event5','event6','event7','event8','event9','event10','event11','event12','event13','event14','event15','event16','event17','event18','event19','event20','event21','event22','event23','event24','event25','event26','event27','event28','event29','event30','event31','event32','event33','event34','event35','event36','event37','event38','event39','event40','event41','event42','event43','event44','event45'],axis=1)
y = df.result
x = df.drop(['result'],axis=1)
df.columns

Index(['event1', 'event2', 'event3', 'blueTopGP', 'blueTopWR', 'blueTopKDA',
       'blueJungleGP', 'blueJungleWR', 'blueJungleKDA', 'blueMidGP',
       'blueMidWR', 'blueMidKDA', 'blueADCGP', 'blueADCWR', 'blueADCKDA',
       'blueSupportGP', 'blueSupportWR', 'blueSupportKDA', 'redTopGP',
       'redTopWR', 'redTopKDA', 'redJungleGP', 'redJungleWR', 'redJungleKDA',
       'redMidGP', 'redMidWR', 'redMidKDA', 'redAdcGP', 'redAdcWR',
       'redAdcKDA', 'redSupportGP', 'redSupportWR', 'redSupportKDA', 'result'],
      dtype='object')

In [3]:
#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'team' column 
encoder_df = pd.DataFrame(encoder.fit_transform(df[['event1', 'event2', 'event3']]).toarray())
encoder_df.columns = encoder.get_feature_names()
#merge one-hot encoded columns back with original DataFrame
transformed_df = df.join(encoder_df)
transformed_df = transformed_df.drop(['event1', 'event2', 'event3'],axis=1)
transformed_df.head()

,blueTopGP,blueTopWR,blueTopKDA,blueJungleGP,blueJungleWR,blueJungleKDA,blueMidGP,blueMidWR,blueMidKDA,blueADCGP,...,x2_BLUE: first_tower_bot,x2_BLUE: first_tower_mid,x2_BLUE: first_tower_top,x2_BLUE: herald,x2_RED: dragon,x2_RED: first_blood,x2_RED: first_tower_bot,x2_RED: first_tower_mid,x2_RED: first_tower_top,x2_RED: herald
0,1,1.00,2.0,1,1.0,2.3,1,1.00,2.7,6,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,0.60,5.6,2,0.0,1.1,9,0.33,1.7,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.00,0.0,1,1.0,3.3,0,0.00,0.0,2,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,23,0.61,3.6,5,0.2,1.6,8,0.63,4.2,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,3,1.00,9.8,2,1.0,7.0,1,0.00,2.0,7,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
corr = transformed_df.corr()

transformed_df[transformed_df.columns[1:]].corr()['result'][:-1].sort_values(ascending=False)

result                      1.000000
x0_RED: first_blood         0.213946
redJungleWR                 0.151312
redMidWR                    0.123204
redAdcWR                    0.111606
redTopWR                    0.108297
redJungleKDA                0.106226
x1_RED: dragon              0.104473
redSupportWR                0.096335
redMidKDA                   0.085499
redAdcKDA                   0.084278
redTopKDA                   0.079186
redSupportGP                0.078713
redJungleGP                 0.071721
x1_RED: herald              0.070720
x0_RED: dragon              0.064645
redMidGP                    0.063792
redSupportKDA               0.061772
redTopGP                    0.056548
x2_RED: first_blood         0.053996
x2_RED: dragon              0.044792
x1_RED: first_blood         0.044293
x2_RED: first_tower_top     0.041218
x2_RED: first_tower_mid     0.041194
x2_RED: first_tower_bot     0.040459
redAdcGP                    0.036972
x1_BLUE: first_tower_bot    0.028019
x

In [6]:
y = transformed_df['result'].copy()
X = transformed_df.drop(['result'],axis=1)

,blueTopGP,blueTopWR,blueTopKDA,blueJungleGP,blueJungleWR,blueJungleKDA,blueMidGP,blueMidWR,blueMidKDA,blueADCGP,...,x2_BLUE: first_tower_bot,x2_BLUE: first_tower_mid,x2_BLUE: first_tower_top,x2_BLUE: herald,x2_RED: dragon,x2_RED: first_blood,x2_RED: first_tower_bot,x2_RED: first_tower_mid,x2_RED: first_tower_top,x2_RED: herald
0,1,1.00,2.0,1,1.00,2.3,1,1.00,2.7,6,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,0.60,5.6,2,0.00,1.1,9,0.33,1.7,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.00,0.0,1,1.00,3.3,0,0.00,0.0,2,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,23,0.61,3.6,5,0.20,1.6,8,0.63,4.2,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,3,1.00,9.8,2,1.00,7.0,1,0.00,2.0,7,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4003,24,0.63,3.6,2,1.00,7.7,12,0.50,7.5,4,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4004,0,0.00,0.0,0,0.00,0.0,5,0.40,3.7,3,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4005,23,0.61,4.6,42,0.48,3.1,8,0.88,8.1,1,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4006,25,0.60,3.2,6,0.83,8.0,8,0.38,4.2,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42, stratify=y)
    return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test = preprocess_input(X,y)

In [8]:
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

models = {
    'Logistic Regression': LogisticRegression(max_iter=10000),
    'Support Vector Machine (Linear Kernel)': LinearSVC(max_iter=100000),
    'Support Vector Machine (RBF Kernel)': SVC(max_iter=100000),
    'Decission Tree': DecisionTreeClassifier(),
    'Adaboost': AdaBoostClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting Classifier': GradientBoostingClassifier()
}


for name, model in models.items():
  model.fit(X_train,y_train)
  print(name + ' trained')

Logistic Regression trained


C:\Users\linco\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Support Vector Machine (Linear Kernel) trained
Support Vector Machine (RBF Kernel) trained
Decission Tree trained
Adaboost trained
Random Forest trained
Gradient Boosting Classifier trained


In [9]:
scores_list = []

for name,model in models.items():    
    scores_list.append({
    'Model': name,
    'Balanced Accuracy': balanced_accuracy_score(y_test,model.predict(X_test)),
    'Precision':  precision_score(y_test,model.predict(X_test)),
    'Recall': recall_score(y_test,model.predict(X_test)),
    'F1-Score': f1_score(y_test,model.predict(X_test)),
    'ROC/AUC Score': roc_auc_score(y_test,model.predict(X_test))
    })
scores = pd.DataFrame(scores_list)

In [10]:
scores

,Model,Balanced Accuracy,Precision,Recall,F1-Score,ROC/AUC Score
0,Logistic Regression,0.663776,0.666667,0.627551,0.646518,0.663776
1,Support Vector Machine (Linear Kernel),0.666439,0.666667,0.637755,0.651890,0.666439
2,Support Vector Machine (RBF Kernel),0.604107,0.614458,0.520408,0.563536,0.604107
3,Decission Tree,0.582827,0.577540,0.551020,0.563969,0.582827
4,Adaboost,0.646254,0.648501,0.607143,0.627141,0.646254
5,Random Forest,0.677358,0.684066,0.635204,0.658730,0.677358
6,Gradient Boosting Classifier,0.662724,0.662234,0.635204,0.648438,0.662724
